<a href="https://colab.research.google.com/github/Maks3174/Phy/blob/main/%D0%94%D0%97_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/House_Rent_Dataset.csv", index_col="Unnamed: 0")

[Інформація про дані](https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset)

In [3]:
df.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2.0,10000.0,1100.0,Ground out of 2,NaN,NaN,Kolkata,Unfurnished,Bachelors/Family,2.0,Contact Owner
1,2022-05-13,2.0,20000.0,800.0,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1.0,NaN
2,2022-05-16,2.0,17000.0,1000.0,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1.0,Contact Owner
3,2022-07-04,2.0,10000.0,800.0,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1.0,Contact Owner
4,NaN,2.0,7500.0,850.0,1 out of 2,Carpet Area,NaN,Kolkata,Unfurnished,Bachelors,1.0,Contact Owner


In [4]:
df = df[['Rent', 'Size', 'Furnishing Status', 'City']]
df.head()

,Rent,Size,Furnishing Status,City
0,10000.0,1100.0,Unfurnished,Kolkata
1,20000.0,800.0,Semi-Furnished,Kolkata
2,17000.0,1000.0,Semi-Furnished,Kolkata
3,10000.0,800.0,Unfurnished,Kolkata
4,7500.0,850.0,Unfurnished,Kolkata


# Завдання 1
Ввиведіть основні числові характеристики даних, розділіть на тренувальну та тестову чатини

In [6]:
print(df.describe())

X = df.drop(columns='Rent')
y = df['Rent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

               Rent         Size
count  4.369000e+03  4402.000000
mean   3.461306e+04   967.214675
std    7.740818e+04   636.118893
min    1.200000e+03    10.000000
25%    1.000000e+04   550.000000
50%    1.600000e+04   850.000000
75%    3.300000e+04  1200.000000
max    3.500000e+06  8000.000000
Training data shape: (3796, 3)
Test data shape: (950, 3)


# Завдання 2
Створіть Pipeline для обробки числових стовпчиків(заповнення пропусків)

In [8]:
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

# Завдання 3
Створіть Pipeline для обробки категоріальних стовпчиків(заповнення пропусків та кодування)

In [11]:
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Завдання 4
Об'єднайте попередні Pipeline в один ColumnTransformer

In [12]:
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, ['Size']),
    ('cat', categorical_pipeline, ['Furnishing Status', 'City'])
])

# Завдання 5
Створіть остаточну модель та натренуйте її

In [18]:
df['Rent'] = df['Rent'].fillna(df['Rent'].mean())

X = df.drop(columns='Rent')
y = df['Rent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Training the model
model_pipeline.fit(X_train, y_train)

Training data shape: (3796, 3)
Test data shape: (950, 3)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Size']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Furnishing Status',
                                                   'City'])])),
                ('regressor', LinearRegression())])

# Завдання 6
Виведіть основні метрики моделі

In [19]:
y_pred = model_pipeline.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R^2 score: {r2}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")

R^2 score: 0.4215888817063431
Mean Absolute Error (MAE): 22217.97557823803
Mean Squared Error (MSE): 2212835939.2616453


# Завдання 7
Збережіть модель, завантажте її та спрогнозуйте ціну автомобіля

In [22]:
joblib.dump(model_pipeline, 'rent_model.pkl')

# Loading the model
loaded_model = joblib.load('rent_model.pkl')

data = pd.DataFrame({'Size': [900],
                     'Furnishing Status': ['Semi-Furnished'],
                     'City': ['Mumbai']})

predicted_rent = loaded_model.predict(data)
print(f"Predicted Rent: {predicted_rent[0]}")

Predicted Rent: 63776.9953286355
